# 一次元CNNによる学習と識別性能評価（生データ&特徴データ）

---
  
引数：raw_tap.csv/raw_rest.csv/TDAvec_autocor_tap.csv/TDAvec_autocor_rest.csvがあるディレクトリまでのパス  
  
---
  
入力： 
* raw_tap.csv/raw_rest.csv  
* TDAvec_autocor_tap.csv/TDAvec_autocor_rest.csv

---
  
出力：ACCURACYpt[loo][k_list]_1dCNN.csv　識別性能評価結果一覧  
k_listはk-分割交差検証法で用いた分割数

---  

生データ（生データ or 生データに移動平均線を適用したデータ）と特徴抽出したデータを一次元CNNを用いて学習し，
交差検証法（k-分割交差検証，leave-one-out交差検証）を用いて識別性能評価を行う．  

ベクトル：ある時刻の各ボクセルの値のパターン


In [1]:
import pandas as pd
import sys

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv1D, UpSampling1D
from keras.layers.pooling import MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


コマンドライン引数でraw_tap.csv/raw_rest.csv/TDAvec_autocor_tap.csv/TDAvec_autocor_rest.csvがあるディレクトリまでのパスを取得

In [2]:
#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../Data_block/analysis_by_programs/20170130ar/12ch/MAL5/'


In [3]:
# RawDataディレクトリかMALディレクトリか識別用
DIRs = PATH.split("/")
DATA_NAME = DIRs[len(DIRs) - 2]

# k-分割交差検証用
k_list = [3, 5, 7]

## ONEdCNN_LOO関数

引数としてTrainingData関数で作成した教師データをX，ラベルをyで受け取る．  
交差検証法の一つleave-one-out交差検証で識別精度評価を行う．

* (1個をテストデータ，残りを教師データにして学習・評価) * すべてのデータ個
* 得られたすべてのデータ個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をTrainingData関数に返す

In [4]:
def ONEdCNN_LOO(X, y):

    # 識別率を格納する配列
    LOOscore = np.zeros(len(X))
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]

    
    # 1個をテストデータ，残りを教師データにして学習・評価
    # すべてのデータに対して行う
    for i in range(len(X)):

        # テストデータ
        X_test = X[i]
        y_test = y[i]

        # テストデータとして使用するデータを除いた教師データを作成
        X_train = np.delete(X, i, 0)
        y_train = np.delete(y, i, 0)


        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))

        # ダミー変数に変換
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test ,2)

        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        LOOscore[i] = score[1]
        

        
    # 評価結果（識別率）の平均を求める
    result = LOOscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print(str(LOOscore) + '\n')

    return result

## ONEdCNN_kCV関数

引数としてTrainingData関数で作成した教師データをX，ラベルをy，データ分割数をkで受け取る．  
交差検証法の一つk-分割交差検証で識別精度評価を行う．

* 学習
* (k分割し，1グループをテストデータ，残りグループを教師データにして評価) * k
* 得られたk個の評価結果（識別率）の平均を求めてパーセントに直す
* 評価結果（識別率）をTrainingData関数に返す

In [5]:
def ONEdCNN_kCV(X, y, k):
    
    
    # 識別率を格納する配列
    kCVscore = np.zeros(k)
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]
    
    # 分割数
    kf = KFold(n_splits=k, shuffle=False)
    # 繰り返し回数
    i = 0

    for train_index, eval_index in kf.split(X):
        
        X_train, X_test = X[train_index], X[eval_index]
        y_train, y_test = y[train_index], y[eval_index]

        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))

        # ダミー変数に変換
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test ,2)

        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=100)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        kCVscore[i] = score[1]
        
        i = i + 1
        

    # 評価結果（識別率）の平均を求める
    result = kCVscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)
    
    print('k = ' + str(k) + '：' + str(kCVscore))

    return result
    

## TrainingData関数 
引数として読み込みたいTapping/Restのそれぞれのファイル名をfile_tap/file_restで受け取る． 
* 機械学習にかけれるようにデータのベクトル化とラベルを作成
* データとラベルを1dCNN_LOO関数，1dCNN_kCV関数に渡す
* 帰ってきた識別率をまとめてmain関数に返す

In [6]:
def TrainingData(file_rest, file_tap):
    
    # 読み込みたいファイルのパス
    PATH_rest = PATH + file_rest
    PATH_tap = PATH + file_tap

    # csvファイル読み込み
    rest = pd.read_csv(PATH_rest, header = 0)
    tap = pd.read_csv(PATH_tap, header = 0) 
    
    # RestとTappingのデータをまとめる
    all_data = pd.concat([rest, tap], axis = 0)

    # ベクトル化
    X = all_data.as_matrix()

    # ラベル作成
    label_rest = np.zeros(len(rest.index))
    label_tap = np.ones(len(tap.index))

    y = np.r_[label_rest, label_tap]

    
    # 学習とleave-one-out交差検証
    
    print('leave-one-out')

    col_name = 'leave-one-out'
    
    #print(col_name)
    
    # SVM_LOO関数
    result_LOO = ONEdCNN_LOO(X, y)
    
    # 評価結果（識別率）をデータフレームに変換・格納
    results = pd.DataFrame({col_name : [result_LOO] })
    
    
    # 学習とk-分割交差検証
    
    print('k-hold Cross-Validation')
    
    # k_listに従って学習・評価
    for i in k_list:
        
        col_name = 'k = ' + str(i)
        
        #print(col_name)
        
        # SVM_CV関数
        result_CV = ONEdCNN_kCV(X, y, i)
        
        # 評価結果（識別率）をデータフレームに変換・格納
        result_CV = pd.DataFrame({col_name : [result_CV] })
        results = pd.concat([results, result_CV], axis = 1)
    
    return results

    

## main関数

In [7]:
if __name__ == '__main__':
    
    # 生データの識別率

    print('--------- ' + DATA_NAME + ' ---------')
    raw_result = TrainingData('raw_rest.csv', 'raw_tap.csv')
    print('\n' + str(raw_result))

    # 特徴抽出データの識別率

    print('\n--------- Feature Extraction Data ---------')
    fe_result = TrainingData('TDAvec_autocor_rest.csv', 'TDAvec_autocor_tap.csv')
    print('\n' + str(fe_result))
    
    # 生データと特徴抽出データの識別率をまとめる
    result_cmp = pd.concat([raw_result, fe_result], axis = 0)
    
    # インデックス名をつける
    result_index = [DATA_NAME, 'TDAvec_autocor']
    result_cmp.index = result_index
    
    # csv書き出し
    PATH_RESULT = PATH + 'ACCURACYpt[loo]' + str(k_list) + '_1dCNN.csv'
    result_cmp.to_csv(PATH_RESULT, index = True)

--------- MAL5 ---------
leave-one-out
Epoch 1/5
111/111 [==============================] - 0s 2ms/step - loss: 0.2437 - acc: 0.6036
Epoch 2/5
111/111 [==============================] - 0s 140us/step - loss: 0.2390 - acc: 0.6757
Epoch 3/5
111/111 [==============================] - 0s 152us/step - loss: 0.2347 - acc: 0.7027
Epoch 4/5
111/111 [==============================] - 0s 150us/step - loss: 0.2301 - acc: 0.7387
Epoch 5/5
1/1 [==============================] - 0s 22ms/step
Epoch 1/5
111/111 [==============================] - 0s 1ms/step - loss: 0.2507 - acc: 0.4144
Epoch 2/5
111/111 [==============================] - 0s 140us/step - loss: 0.2485 - acc: 0.5495
Epoch 3/5
111/111 [==============================] - 0s 142us/step - loss: 0.2467 - acc: 0.6486
Epoch 4/5
111/111 [==============================] - 0s 144us/step - loss: 0.2452 - acc: 0.7027
Epoch 5/5
1/1 [==============================] - 0s 36ms/step
Epoch 1/5
111/111 [==============================] - 0s 2ms/step - loss: 

111/111 [==============================] - 0s 167us/step - loss: 0.2168 - acc: 0.7928
Epoch 4/5
111/111 [==============================] - 0s 167us/step - loss: 0.2135 - acc: 0.7838
Epoch 5/5
1/1 [==============================] - 0s 276ms/step
Epoch 1/5
111/111 [==============================] - 1s 6ms/step - loss: 0.2305 - acc: 0.7748
Epoch 2/5
111/111 [==============================] - 0s 175us/step - loss: 0.2265 - acc: 0.8108
Epoch 3/5
111/111 [==============================] - 0s 227us/step - loss: 0.2226 - acc: 0.8108
Epoch 4/5
111/111 [==============================] - 0s 182us/step - loss: 0.2194 - acc: 0.8198
Epoch 5/5
1/1 [==============================] - 0s 274ms/step
Epoch 1/5
111/111 [==============================] - 1s 6ms/step - loss: 0.2379 - acc: 0.5495
Epoch 2/5
111/111 [==============================] - 0s 177us/step - loss: 0.2329 - acc: 0.6126
Epoch 3/5
111/111 [==============================] - 0s 181us/step - loss: 0.2293 - acc: 0.6757
Epoch 4/5
111/111 [=====

1/1 [==============================] - 1s 524ms/step
Epoch 1/5
111/111 [==============================] - 1s 11ms/step - loss: 0.2872 - acc: 0.5135
Epoch 2/5
111/111 [==============================] - 0s 159us/step - loss: 0.2814 - acc: 0.5495
Epoch 3/5
111/111 [==============================] - 0s 167us/step - loss: 0.2745 - acc: 0.5676
Epoch 4/5
111/111 [==============================] - 0s 155us/step - loss: 0.2682 - acc: 0.6126
Epoch 5/5
1/1 [==============================] - 1s 517ms/step
Epoch 1/5
111/111 [==============================] - 1s 11ms/step - loss: 0.2166 - acc: 0.5135
Epoch 2/5
111/111 [==============================] - 0s 187us/step - loss: 0.2137 - acc: 0.5946
Epoch 3/5
111/111 [==============================] - 0s 179us/step - loss: 0.2110 - acc: 0.6486
Epoch 4/5
111/111 [==============================] - 0s 158us/step - loss: 0.2085 - acc: 0.7117
Epoch 5/5
1/1 [==============================] - 1s 524ms/step
Epoch 1/5
111/111 [==============================] - 1s

111/111 [==============================] - 2s 15ms/step - loss: 0.2715 - acc: 0.4955
Epoch 2/5
111/111 [==============================] - 0s 204us/step - loss: 0.2581 - acc: 0.4955
Epoch 3/5
111/111 [==============================] - 0s 198us/step - loss: 0.2448 - acc: 0.5045
Epoch 4/5
111/111 [==============================] - 0s 205us/step - loss: 0.2341 - acc: 0.5856
Epoch 5/5
1/1 [==============================] - 1s 756ms/step
Epoch 1/5
111/111 [==============================] - 2s 15ms/step - loss: 0.2653 - acc: 0.4955
Epoch 2/5
111/111 [==============================] - 0s 142us/step - loss: 0.2590 - acc: 0.4955
Epoch 3/5
111/111 [==============================] - 0s 141us/step - loss: 0.2520 - acc: 0.4955
Epoch 4/5
111/111 [==============================] - 0s 183us/step - loss: 0.2455 - acc: 0.4955
Epoch 5/5
1/1 [==============================] - 1s 850ms/step
Epoch 1/5
111/111 [==============================] - 2s 17ms/step - loss: 0.2532 - acc: 0.4414
Epoch 2/5
111/111 [====

111/111 [==============================] - 0s 221us/step - loss: 0.2242 - acc: 0.7387
Epoch 4/5
111/111 [==============================] - 0s 198us/step - loss: 0.2205 - acc: 0.7838
Epoch 5/5
1/1 [==============================] - 1s 966ms/step
Epoch 1/5
111/111 [==============================] - 2s 21ms/step - loss: 0.2890 - acc: 0.4955
Epoch 2/5
111/111 [==============================] - 0s 212us/step - loss: 0.2820 - acc: 0.4955
Epoch 3/5
111/111 [==============================] - 0s 207us/step - loss: 0.2751 - acc: 0.4955
Epoch 4/5
111/111 [==============================] - 0s 247us/step - loss: 0.2685 - acc: 0.4955
Epoch 5/5
1/1 [==============================] - 1s 1s/step
Epoch 1/5
111/111 [==============================] - 3s 23ms/step - loss: 0.2829 - acc: 0.4955
Epoch 2/5
111/111 [==============================] - 0s 240us/step - loss: 0.2771 - acc: 0.4865
Epoch 3/5
111/111 [==============================] - 0s 242us/step - loss: 0.2718 - acc: 0.5045
Epoch 4/5
111/111 [======

...

Epoch 3/5
19/19 [==============================] - 0s 721us/step - loss: 0.3201 - acc: 0.5263
Epoch 4/5
19/19 [==============================] - 0s 682us/step - loss: 0.3058 - acc: 0.5263
Epoch 5/5
1/1 [==============================] - 2s 2s/step
Epoch 1/5
19/19 [==============================] - 5s 286ms/step - loss: 0.2756 - acc: 0.5789
Epoch 2/5
19/19 [==============================] - 0s 651us/step - loss: 0.2739 - acc: 0.5789
Epoch 3/5
19/19 [==============================] - 0s 583us/step - loss: 0.2718 - acc: 0.6316
Epoch 4/5
19/19 [==============================] - 0s 674us/step - loss: 0.2698 - acc: 0.5789
Epoch 5/5
1/1 [==============================] - 3s 3s/step
Epoch 1/5
19/19 [==============================] - 6s 296ms/step - loss: 0.4589 - acc: 0.5263
Epoch 2/5
19/19 [==============================] - 0s 739us/step - loss: 0.4534 - acc: 0.5263
Epoch 3/5
19/19 [==============================] - 0s 794us/step - loss: 0.4461 - acc: 0.5263
Epoch 4/5
19/19 [=================

4/4 [==============================] - 3s 671ms/step
Epoch 1/5
16/16 [==============================] - 6s 356ms/step - loss: 0.3090 - acc: 0.6250
Epoch 2/5
16/16 [==============================] - 0s 696us/step - loss: 0.2946 - acc: 0.6250
Epoch 3/5
16/16 [==============================] - 0s 783us/step - loss: 0.2808 - acc: 0.5625
Epoch 4/5
16/16 [==============================] - 0s 729us/step - loss: 0.2701 - acc: 0.5625
Epoch 5/5
4/4 [==============================] - 3s 741ms/step
Epoch 1/5
16/16 [==============================] - 6s 378ms/step - loss: 0.2574 - acc: 0.4375
Epoch 2/5
16/16 [==============================] - 0s 799us/step - loss: 0.2561 - acc: 0.4375
Epoch 3/5
16/16 [==============================] - 0s 785us/step - loss: 0.2541 - acc: 0.4375
Epoch 4/5
16/16 [==============================] - 0s 730us/step - loss: 0.2523 - acc: 0.4375
Epoch 5/5
4/4 [==============================] - 3s 662ms/step
k = 5：[ 0.    0.    0.5   0.    0.75]
Epoch 1/5
17/17 [==============

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-898798b0f739>", line 12, in <module>
    fe_result = TrainingData('TDAvec_autocor_rest.csv', 'TDAvec_autocor_tap.csv')
  File "<ipython-input-6-5ba7b09cde04>", line 51, in TrainingData
    result_CV = ONEdCNN_kCV(X, y, i)
  File "<ipython-input-5-8b9e6d517828>", line 50, in ONEdCNN_kCV
    score = model.evaluate(X_test, y_test)
  File "/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/keras/models.py", line 922, in evaluate
    sample_weight=sample_weight)
  File "/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/keras/engine/training.py", line 1692, in evaluate
    steps=steps)
  File "/Users/nakayamarina/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/keras/

KeyboardInterrupt: 